In this notebook, we train character-based LSTM models to generate Shakespearean-style sonnets and tune various parameters.

In [1]:
import numpy as np

from DataProcessing import *
from RNNProcessing import *

import keras
from keras.callbacks import EarlyStopping, LambdaCallback, ModelCheckpoint

Using TensorFlow backend.


## With punctuation and newlines
Read in Shakespearean sonnets with all punctuation and newlines preserved, enabling learning of the natural poem structure.

In [2]:
# Load data
sonnets, syllableDic = loadShake_char(stripPunc=False)
X_code, y_code, char2code, code2char = text2seq(sonnets)
X, y = seq2cat(X_code, y_code, len(char2code))

Define callback function for use in training.

In [3]:
def epoch_lines(epoch, _):
    # Generate a sequence of lines after each epoch
    print('--------------- Epoch %d Completed ---------------' % epoch)
    seed = X_code[np.random.choice(len(X_code))]
    T = 1
    seq_out = gen_lines(model, seed, code2char, n_lines=2, T=T, verbose=False)
    print(seq_out + '\n')

lines_callback = LambdaCallback(on_epoch_end=epoch_lines)

Train model with varying numbers of neurons and layers, using loss at 20 epochs for comparison.

In [4]:
# Define architectures to consider
n_layers = [1, 1, 1, 2, 2]
n_units = [[64], [128], [256], [64, 64], [128, 128]]
losses = np.zeros(len(n_layers))

for i in range(len(n_layers)):
    # Construct model
    model = build_model_LSTM(X.shape[1:], y.shape[1],
                             n_layers=n_layers[i], n_units=n_units[i])
    model.summary()
    
    # Set model checkpoint
    file = './TrainingTemp/RNN_char-LSTM_arch{:d}.h5'.format(i)
    model_checkpoint = ModelCheckpoint(file,
                                       monitor='loss', save_best_only=True)

    # Compile and fit model
    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])
    fit = model.fit(X, y, batch_size=64, epochs=20,
                    callbacks=[model_checkpoint],
                    verbose=1)
    losses[i] = model.evaluate(X, y)[0]
    
ind = np.argmin(losses)
print(ind)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                26368     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                2470      
Total params: 28,838
Trainable params: 28,838
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/20
88129/88129 [==============================] - 117s 1ms/step - loss: 2.5979 - acc: 0.2655
Epoch 2/20
88129/88129 [==============================] - 107s 1ms/step - loss: 2.1666 - acc: 0.3639
Epoch 3/20
88129/88129 [==============================] - 108s 1ms/step - loss: 2.0313 - acc: 0.3992
Epoch 4/20
88129/88129 [==============================] - 107s 1ms/step - loss: 1.9345 - acc: 0.4222
Epoch 5/20

Epoch 17/20
88129/88129 [==============================] - 109s 1ms/step - loss: 0.7906 - acc: 0.7493
Epoch 18/20
88129/88129 [==============================] - 109s 1ms/step - loss: 0.7460 - acc: 0.7649
Epoch 19/20
88129/88129 [==============================] - 109s 1ms/step - loss: 0.7104 - acc: 0.7746
Epoch 20/20
88129/88129 [==============================] - 88s 1ms/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 40, 64)            26368     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_4 (Dense)              (None, 38)                2470      
Total params: 61,862
Trainable params: 61,862
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
88129/88129 [

We then train the best architecture for 200 epochs, saving models with best validation or training losses.

In [5]:
# Construct model
model = build_model_LSTM(X.shape[1:], y.shape[1],
                         n_layers=n_layers[ind], n_units=n_units[ind])
model.summary()

# Add callback functions
model_checkpoint_v = ModelCheckpoint('./TrainingTemp/RNN_char-LSTM_valid.h5',
                                     monitor='val_loss', save_best_only=True)
model_checkpoint_t = ModelCheckpoint('./TrainingTemp/RNN_char-LSTM_train.h5',
                                     monitor='loss', save_best_only=True)

# Compile and fit model
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
fit = model.fit(X, y, batch_size=64, epochs=200,
                validation_split=0.2,
                callbacks=[model_checkpoint_v, model_checkpoint_t,
                           lines_callback],
                verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 256)               302080    
_________________________________________________________________
dense_6 (Dense)              (None, 38)                9766      
Total params: 311,846
Trainable params: 311,846
Non-trainable params: 0
_________________________________________________________________
Train on 70503 samples, validate on 17626 samples
Epoch 1/200
70503/70503 [==============================] - 97s 1ms/step - loss: 2.4311 - acc: 0.3036 - val_loss: 2.1204 - val_acc: 0.3729
--------------- Epoch 0 Completed ---------------
ing thes and
woth f allige,

Epoch 2/200
70503/70503 [==============================] - 96s 1ms/step - loss: 1.9838 - acc: 0.4141 - val_loss: 1.9227 - val_acc: 0.4260
--------------- Epoch 1 Completed ---------------
me ithor thonghoud iving, beave  in,
roth lover enise.

Epoch 3/200
70503/705

70503/70503 [==============================] - 96s 1ms/step - loss: 0.3370 - acc: 0.8983 - val_loss: 2.9923 - val_acc: 0.4292
--------------- Epoch 30 Completed ---------------
 blood,
poing of love taul my decay trink mether,

Epoch 32/200
70503/70503 [==============================] - 96s 1ms/step - loss: 0.3331 - acc: 0.8982 - val_loss: 3.0366 - val_acc: 0.4332
--------------- Epoch 31 Completed ---------------
d many give change is thy sorfow, nowers mine,
the scance mones, ort thy sab' of content.

Epoch 33/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.3167 - acc: 0.9043 - val_loss: 3.0657 - val_acc: 0.4332
--------------- Epoch 32 Completed ---------------
t the penfacuriggance alone deceives,
but lake even ived he winds in your did,

Epoch 34/200
70503/70503 [==============================] - 96s 1ms/step - loss: 0.3043 - acc: 0.9075 - val_loss: 3.1230 - val_acc: 0.4273
--------------- Epoch 33 Completed ---------------
i see twe by that summer than b

70503/70503 [==============================] - 96s 1ms/step - loss: 0.1815 - acc: 0.9430 - val_loss: 3.8322 - val_acc: 0.4271
--------------- Epoch 61 Completed ---------------
ve there bredst?
  that macking honered with trultasked ach:

Epoch 63/200
70503/70503 [==============================] - 96s 1ms/step - loss: 0.1694 - acc: 0.9482 - val_loss: 3.8598 - val_acc: 0.4267
--------------- Epoch 62 Completed ---------------
aviour, beauty doth he give
and with atternal lure in my should by all are,

Epoch 64/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.1748 - acc: 0.9454 - val_loss: 3.8942 - val_acc: 0.4251
--------------- Epoch 63 Completed ---------------
d surmer,
whose like here whils no hours and there cantler,

Epoch 65/200
70503/70503 [==============================] - 96s 1ms/step - loss: 0.1786 - acc: 0.9437 - val_loss: 3.8944 - val_acc: 0.4260
--------------- Epoch 64 Completed ---------------
hee, welculies this, thou dost stay,
no longers their

--------------- Epoch 93 Completed ---------------
ong,
when you have lives i vind, yet can their right,

Epoch 95/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.1321 - acc: 0.9581 - val_loss: 4.2127 - val_acc: 0.4293
--------------- Epoch 94 Completed ---------------
 hug?ance love,
that haw speaking have, my heart his growth

Epoch 96/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.1421 - acc: 0.9537 - val_loss: 4.2149 - val_acc: 0.4286
--------------- Epoch 95 Completed ---------------
 do with me remain,
without thy help, i am i will heart his shyme

Epoch 97/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.1381 - acc: 0.9551 - val_loss: 4.2283 - val_acc: 0.4291
--------------- Epoch 96 Completed ---------------
ty's beties.
these present-absent with swift motion slide.

Epoch 98/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.1286 - acc: 0.9604 - val_loss: 4.2682 - val_acc: 0.42

70503/70503 [==============================] - 95s 1ms/step - loss: 0.1103 - acc: 0.9656 - val_loss: 4.4037 - val_acc: 0.4335
--------------- Epoch 125 Completed ---------------
then believe me, know my dut, since minetest take.
  yet him what alone, that enfulful as faulest trath.

Epoch 127/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.1119 - acc: 0.9640 - val_loss: 4.3988 - val_acc: 0.4303
--------------- Epoch 126 Completed ---------------
lutast with think on thee, and deemst commerse,
mornst by this summer oncy his most comp,

Epoch 128/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.1079 - acc: 0.9664 - val_loss: 4.3880 - val_acc: 0.4323
--------------- Epoch 127 Completed ---------------
nstant stross of that fair thou ow'st,
no lifterest ampear ncas when thou art gone,

Epoch 129/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.1186 - acc: 0.9619 - val_loss: 4.4147 - val_acc: 0.4285
--------------- 

my sin you did impute,
which shall be framene on the were where know,

Epoch 158/200
70503/70503 [==============================] - 97s 1ms/step - loss: 0.0972 - acc: 0.9690 - val_loss: 4.5470 - val_acc: 0.4296
--------------- Epoch 157 Completed ---------------
y long heart's face,
to truth mis living for my stain,

Epoch 159/200
70503/70503 [==============================] - 97s 1ms/step - loss: 0.1009 - acc: 0.9680 - val_loss: 4.5339 - val_acc: 0.4325
--------------- Epoch 158 Completed ---------------
g on dear my tongue that thou dostill.
  then that thy shalow the story speed not fight,

Epoch 160/200
70503/70503 [==============================] - 97s 1ms/step - loss: 0.0885 - acc: 0.9725 - val_loss: 4.5426 - val_acc: 0.4336
--------------- Epoch 159 Completed ---------------
 the best of love,
thy shape with plyanse moun, thou shouldst in me,

Epoch 161/200
70503/70503 [==============================] - 97s 1ms/step - loss: 0.0982 - acc: 0.9688 - val_loss: 4.5538 - val_acc: 0.42

70503/70503 [==============================] - 96s 1ms/step - loss: 0.0777 - acc: 0.9760 - val_loss: 4.6273 - val_acc: 0.4340
--------------- Epoch 188 Completed ---------------
thus have i had thee as a drow appear,
that you your sempined in the judgled with flate!

Epoch 190/200
70503/70503 [==============================] - 96s 1ms/step - loss: 0.0836 - acc: 0.9738 - val_loss: 4.6585 - val_acc: 0.4328
--------------- Epoch 189 Completed ---------------
 sweetest this love, thou mayst be fail,
that you your tenureful as your most he tome,

Epoch 191/200
70503/70503 [==============================] - 96s 1ms/step - loss: 0.0896 - acc: 0.9713 - val_loss: 4.6563 - val_acc: 0.4300
--------------- Epoch 190 Completed ---------------
 me tongue-tied speaking of earth,
summers steal themselvess shook my fate,

Epoch 192/200
70503/70503 [==============================] - 95s 1ms/step - loss: 0.0848 - acc: 0.9732 - val_loss: 4.6374 - val_acc: 0.4305
--------------- Epoch 191 Completed -------

Generate example text from model with lowest validation loss.

In [6]:
# Load model
model = keras.models.load_model('./TrainingTemp/RNN_char-LSTM_valid.h5')

# Choose seed
seed = X_code[np.random.choice(len(X_code))]
print('Seed Sequence:')
print(''.join([code2char[code] for code in seed]))
print('')

# Generate sequences for different temperatures
for T in [0.5, 1, 1.5]:
    seq_out = gen_lines(model, seed, code2char, n_lines=14, T=T,
                        verbose=False)
    print('Output Sequence for Temperature %.1f:' % T)
    print(seq_out)
    print('')

Seed Sequence:
 on now,
will be a tattered weed of smal

Output Sequence for Temperature 0.5:
l stronger.
  the cream for love the world doth beauty,
and the sweet self in the self the sweet,
but the can a conterme time from the meard.
  for thou art the strong and so all the cheek,
  but that the live that thou art beauty,
but that that the summer strange he sum the cloved
the crease the stard the star thy self the strong,
that i spor thy self in his spring of with,
which thou art the rast of with the strange,
the can the summer's leasure love and the the thee,
which in their praise hath that lears the still doth stend:
than the warling all be this so recoint,
and you are the changar have a well on me?

Output Sequence for Temperature 1.0:
 in heart,
sur ont those ray, beauty place with tomb
i an eye anded summer, be time is tange,
every ome anwerat with beauty with destopy,
framkle to yewione eills is all thy wrot,
but write's respice all offuch excemune o'er

to grast the blay with 

Generate example text from model with lowest training loss.

In [7]:
# Load model
model = keras.models.load_model('./TrainingTemp/RNN_char-LSTM_train.h5')

# Choose seed
seed = X_code[np.random.choice(len(X_code))]
print('Seed Sequence:')
print(''.join([code2char[code] for code in seed]))
print('')

# Generate sequences for different temperatures
for T in [0.5, 1, 1.5]:
    seq_out = gen_lines(model, seed, code2char, n_lines=14, T=T,
                        verbose=False)
    print('Output Sequence for Temperature %.1f:' % T)
    print(seq_out)
    print('')

Seed Sequence:
 the place where he would be.
but ah, th

Output Sequence for Temperature 0.5:
emselves and i braish (to you,
  as he takes where thou art altewattion
the frese wherein they see doth show it dead.
  yet simpect telfure to me than gitder pent,
the roses filler things but inderit cloase,
three but a till the world, upbest bud now,
  so summer in my thoughts (yet were distraine.
the course undait doth that proud to be
and that repost doth pregsain of a confound.
hather i by this sun as thou shalt still,
they looks gives the eyes of houre i supp,
for thrie part in my thou becter (doth lie,
make so mary of thine age such sweetest,
by unions minutest in therefore to be woe,

Output Sequence for Temperature 1.0:
ememing dubling strong swaited,
but shall neger liveless deep adeftiques.
so all my argume that me ton so riss,
since shade themselves assury as me with place,
and for my self mine own worth do define,
as i all other time mo handy seem days,
even that your loy puncelest